In [1]:
import requests
import json
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC  #importing machine learning classification algorithm
import random
import pickle
import nltk

### Mini dataset

In [27]:

responses = {0: { "intent":"greetings","response":["Welcome to Stark Industries","Hi there","Hello, What can I do for you?",'jocasta here!!']},
             1: {"intent" : "about","response":['Stark Industries is owned by Tony stark and run by Pepper Potts as you know the Iron Man itself and my boss','Stark Industries manufactures weapons and arm forces for US Military']},
             2: {"intent":"owns","response":['Solely by Pepper Potts the one and only wife on Mr.Tony Stark my boss']},
             3: {"intent":"dead","response":['Tony Stark is not dead in this reality this universe is not 616 we are still having him here']},
             4: {"intent":"careers",'response':['We are currently looking for \n1: Data Scientist \n2: Ph.D in Artificial Intelligence \n3: Civil Engineer \n4:Mechanical Engineer \n5: Also freshers who want to learn more in AI and ML']},
            5: {"intent":"covid-19",'response':['Our scientist are building vacines on the headache virus by China','Mr.Bruce Banner is helping us to build vacines for corona virus','Many scientists around the world are working with us for building vacines','Soon dispatching large amounts of vacines in air to remove corona']},
            6: {"intent":"apply","response":['You can apply for jobs by sending us an email on careers@starkindustries.com and we will soon reach out for further process','send your resume on careers@starkindustries.com and we will soon reach out for further process','You can mails us at careers@starkindustries.com and we will soon reach out for further process']},
            7:{"intent":"farewell",'response':['Stay Safe, Stay Home Bye','Bye bye!! hope you had good time talking with me','Good Bye!! have a nice day','Farewell, until next time','Ask me anytime, Josacta is always here']},
            8:{"intent":"out_of_bounds","response":['Sorry, but I am not Jarvis so can\'t understand most of your feelings','can you say it properly I wasn\'t able to understand you','Sorry i wasn\'t able to undertsand']}}

In [38]:
# training data
data = [["hi",0],["hello",0],["hi jocasta",0],['hi there',0],['what is stark industries',1],['about stark industries',1],['stark industries',1],['who owns stark industries',2]
        ,['is tony stark dead?',3],['Is tony alive?',3],['tony stark',3],['tony stark alive or dead',3],['is job availble for me?',4],['is job available',4],['i am a fresher',4],['fresher can get this',4],['freshers',4],['careers in stark industries',4]
        ,['is stark industries recruting new employees?',4],['jobs in your company',4],['about covid-19',5],['vacines for coronavirus',5]
        ,['is stark Industries building vacines for corona virus',5],['corona virus',5],['is your company building vacines for covid-19',5]
        ,['apply for job',6],['where to apply for job?',6],['where to submit resume for job?',6],['how can I apply for job in your company',6]
        ,['can you tell me where can I apply for job in your company',6]
        ,['bye',7],['good-bye',7],['farewell',7],['now i am going',7],['see you',7],['untill next time',7],['alright then bye!',7]]

In [39]:
df = pd.DataFrame(data, columns = ['text', 'intent'])

In [40]:
df

,text,intent
0,hi,0
1,hello,0
2,hi jocasta,0
3,hi there,0
4,what is stark industries,1
5,about stark industries,1
6,stark industries,1
7,who owns stark industries,2
8,is tony stark dead?,3
9,Is tony alive?,3


In [46]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [41]:
lemmer = nltk.stem.WordNetLemmatizer()

In [42]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [43]:
def Normalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [44]:
x = df['text']
y = df['intent']

In [47]:
vectorizer =  TfidfVectorizer(tokenizer=Normalize, stop_words='english')
X = vectorizer.fit_transform(x) 

e:\anaconda3\envs\madbot\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [48]:
clf = LinearSVC(max_iter=10000 , C= 0.1)

In [49]:
clf.fit(X,y)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [50]:
text_test = ['wa']

In [51]:
X_test = vectorizer.transform(text_test)

In [52]:
prediction = clf.predict(X_test)
clf.decision_function(X_test)

array([[-0.62880777, -0.73937179, -0.78734205, -0.66907788, -0.48502205,
        -0.58712748, -0.63901833, -0.41975879]])

In [53]:
prediction[0]

7

In [54]:
pickle.dump(clf,open("model_jocasta.pkl", 'wb'))

In [55]:
load_model = pickle.load(open("model_jocasta.pkl", 'rb'))

In [56]:
load_model.predict(X_test)

array([7], dtype=int64)

In [57]:
def response(user_response):
    text_test = [user_response]
    X_test = vectorizer.transform(text_test)
    prediction = clf.predict(X_test)
    print(prediction)
    reply = random.choice(responses[prediction[0]]["response"])
    return reply

In [58]:
class telegram_bot():
    def __init__(self):
        self.token = ""    #write your token here!
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)

    def send_message(self,msg,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={msg}"
        if msg is not None:
            requests.get(url)

    def grab_token(self):
        return tokens


In [80]:
from tabulate import tabulate
data = pd.read_csv('diamonds.csv',index_col=0)
data

,shape,carat,cut,color,clarity,depth,table,price($)
id,,,,,,,,
1,princess,0.23,Ideal,E,SI2,61.5,55.0,326
2,round,0.21,Premium,E,SI1,59.8,61.0,326
3,round,0.23,Good,E,VS1,56.9,65.0,327
4,princess,0.29,Premium,I,VS2,62.4,58.0,334
5,round,0.31,Good,J,SI2,63.3,58.0,335
...,...,...,...,...,...,...,...,...
95,princess,0.78,Very Good,G,SI2,63.8,56.0,2759
96,princess,0.70,Good,E,VS2,57.5,58.0,2759
97,round,0.70,Good,F,VS1,59.4,62.0,2759


In [108]:
round = data
round.query('shape == "round"',inplace=True)
list1 = round.values.tolist()


In [114]:
str1 = ''
for i,j in enumerate(list1):
    str1 += str(i+1)+"). "+str(j[0])+" -- "+str(j[1]) + "(caret) -- "+str(j[2])+ "(cut) -- $"+str(j[5])+"\n"
    
str1

'1). round -- 0.21(caret) -- Premium(cut) -- $59.8\n2). round -- 0.23(caret) -- Good(cut) -- $56.9\n3). round -- 0.31(caret) -- Good(cut) -- $63.3\n4). round -- 0.24(caret) -- Very Good(cut) -- $62.3\n5). round -- 0.22(caret) -- Fair(cut) -- $65.1\n6). round -- 0.3(caret) -- Good(cut) -- $64.0\n7). round -- 0.32(caret) -- Premium(cut) -- $60.9\n8). round -- 0.3(caret) -- Good(cut) -- $63.4\n9). round -- 0.3(caret) -- Good(cut) -- $63.3\n10). round -- 0.23(caret) -- Very Good(cut) -- $63.8\n11). round -- 0.31(caret) -- Very Good(cut) -- $59.4\n12). round -- 0.23(caret) -- Very Good(cut) -- $60.4\n13). round -- 0.3(caret) -- Very Good(cut) -- $62.2\n14). round -- 0.23(caret) -- Very Good(cut) -- $60.9\n15). round -- 0.23(caret) -- Very Good(cut) -- $61.9\n16). round -- 0.23(caret) -- Good(cut) -- $64.1\n17). round -- 0.33(caret) -- Ideal(cut) -- $61.8\n18). round -- 0.33(caret) -- Ideal(cut) -- $61.2\n19). round -- 0.26(caret) -- Good(cut) -- $65.2\n20). round -- 0.32(caret) -- Good(cut)

In [115]:
tbot = telegram_bot()

update_id = None

def make_reply(msg):
    if msg is not None:
        if msg == '/start':
            reply = 'Jocasta Here I will be helping you with the following information: \nAbout Stark Industries \nCovid-19 \n Careers \nHow to apply for job'
        elif msg == '/stop':
            reply = 'See you Bye!!'
        elif msg == 'round':
            reply =">\n" + str1 + ">"
        else:
            reply = response(msg)
   
    return reply

while True:
    print("...")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    print(updates)
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)
            reply = make_reply(message)
            tbot.send_message(reply,from_)

...
[{'update_id': 176989951, 'message': {'message_id': 140, 'from': {'id': 702314504, 'is_bot': False, 'first_name': 'Deathstroke', 'username': 'Deathstrokehell', 'language_code': 'en'}, 'chat': {'id': 702314504, 'first_name': 'Deathstroke', 'username': 'Deathstrokehell', 'type': 'private'}, 'date': 1592810405, 'text': 'round'}}]
176989951
round
702314504
...
[]
...
[]
...
[]
...
[]
...
[]
...
[]
...
[]
...


KeyboardInterrupt: 